In [1]:
import h5py
import numpy as np
import pandas as pd
from core_functions import call_disort
from tqdm import tqdm 

In [2]:
# must be 1.26.4
np.__version__

'1.26.4'

In [3]:
!gcc --version
!gfortran --version

# must be 3.12.4
!python --version

gcc (MinGW-W64 x86_64-ucrt-posix-seh, built by Brecht Sanders, r3) 14.1.0
Copyright (C) 2024 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

GNU Fortran (MinGW-W64 x86_64-ucrt-posix-seh, built by Brecht Sanders, r3) 14.1.0
Copyright (C) 2024 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

Python 3.12.4


In [7]:
GAS_EXT_PATH = 'gas_ext/'
GAS_EXT_FILE = 'gas_ext_CO2_hot_ames_bezard_H2O_HDO_hitran_foreign_cont_MTCKD_005_9600_11000_104.csv'

PROFILE_FILENAME = 'VIRAPROFILE.txt'
AEROSOL_FILENAME = 'venusclouds_75_Haus16.txt'

LINES_PATH = 'databases/'
INPUT_PATH = 'input/'

CORE_NAME = GAS_EXT_FILE[8:-4]

OUTPUT_PATH = f'disort_output/'
OUTPUT_FILENAME = f'disort_table_{CORE_NAME}.h5'

In [8]:
OUTPUT_FILENAME

'disort_table_CO2_hot_ames_bezard_H2O_HDO_hitran_foreign_cont_MTCKD_005_9600_11000_104.h5'

In [9]:
gas_abs_data = pd.read_csv(GAS_EXT_PATH + GAS_EXT_FILE, header=None)
wavenum = np.array(gas_abs_data)[0]

In [10]:
# Определим сетки параметров
sf_ = np.arange(2.4, 0.3, -0.4)  # sf (2.4, 2.0, 1.6, 1.2, 0.8, 0.4)
hmin_ = np.arange(-3, 11, 1)  # hmin (-3, -2, -1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10 )
emis_ang_ = np.arange(0, 30, 10) # emis ang (0, 10, 20)

# sf_ = np.array([1.6, 0.8])
# hmin_ = np.array([-3, 0, 3, 6, 9])
# emis_ang_ = np.array([0, 5]) 
# 0.7660,  0.8660,  0.9397,  0.9848, 1.0
# Длина спектра, который возвращает call_disort
spectrum_length = len(wavenum)

In [ ]:
# Создаем или открываем HDF5 файл для записи спектров
with h5py.File(OUTPUT_PATH + OUTPUT_FILENAME, 'a') as f:
    # Если это первый запуск, создаем датасеты (если уже созданы, просто загружаем их)
    if 'sf' not in f:
        f.create_dataset('sf', data=sf_)
        f.create_dataset('topo', data=hmin_)
        f.create_dataset('emis_ang', data=emis_ang_)
        spectra_dataset = f.create_dataset('spectra', 
                                           shape=(len(sf_), len(hmin_), len(emis_ang_), spectrum_length),
                                           dtype='f4')
        progress_dataset = f.create_dataset('progress', data=np.zeros((len(sf_), len(hmin_), len(emis_ang_))))
    else:
        spectra_dataset = f['spectra']
        progress_dataset = f['progress']
        # print(progress_dataset)

    # Общее количество комбинаций для отображения прогресса
    total_steps = len(sf_) * len(hmin_) * len(emis_ang_)

    # Инициализируем прогресс-бар с помощью tqdm
    with tqdm(total=total_steps, desc="Заполнение HDF5 файла", unit="combination") as pbar:
        for i, x1 in enumerate(sf_):
            for j, x2 in enumerate(hmin_):
                for k, x3 in enumerate(emis_ang_):
                    # Проверяем, был ли уже записан спектр для этой комбинации параметров
                    if progress_dataset[i, j, k] == 1:
                        pbar.update(1)
                        continue  # Пропускаем, если уже записано
                    
                    # Вызываем функцию для расчета спектра
                    _, spectrum = call_disort(
                        lines_path=LINES_PATH, 
                        input_path=INPUT_PATH, 
                        profile_filename=PROFILE_FILENAME, 
                        aerosol_filename=AEROSOL_FILENAME, 
                        gas_ext_path=GAS_EXT_PATH, 
                        gas_ext_file=GAS_EXT_FILE, 
                        hmin=x2, 
                        emis_ang=x3,
                        sf=x1, 
                        emis=0.95, 
                        ic=0.3, 
                        dh=0
                    )
                    
                    # Записываем спектр в соответствующую позицию датасета
                    spectra_dataset[i, j, k, :] = spectrum
                    
                    # Обновляем прогресс, указывая, что спектр записан
                    progress_dataset[i, j, k] = 1
                    
                    # Обновляем прогресс-бар
                    pbar.update(1)

print("Заполнение HDF5 файла завершено.")

Заполнение HDF5 файла:  67%|█████████████████████████████▌              | 169/252 [2:47:15<1:19:37, 57.57s/combination]